In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install transformers
!pip3 install pytorch_lightning
!pip3 install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.1 MB/s 
     |████████████████████████████████| 7.6 MB 53.7 MB/s 
     |████████████████████████████████| 182 kB 76.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 7.8 MB/s 
     |████████████████████████████████| 125 kB 71.7 MB/s 
     |████████████████████████████████| 512 kB 63.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from transformers import logging
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
import io
import torchmetrics
from google.colab import files

In [ ]:
logging.set_verbosity_error()

RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)

train = files.upload()
test = files.upload()

INFO:lightning_lite.utilities.seed:Global seed set to 42


Saving train.csv to train.csv


Saving test.csv to test.csv


In [ ]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
#val_df = pd.read_csv('val.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

train_df, val_df = train_test_split(train_df, test_size=0.05)

LABEL_COLUMNS = train_df.columns.tolist()[1:]

BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
model = BertModel.from_pretrained(BERT_MODEL_NAME)

MAX_TOKEN_COUNT = 512

auroc = torchmetrics.AUROC(task="binary")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
class SensetiveData(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int = 128):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        comment_text = data_row.text
        labels = data_row[LABEL_COLUMNS]

        encoding = self.tokenizer.encode_plus(
        comment_text,
        add_special_tokens=True,
        max_length=self.max_token_len,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
        )

        return dict(
        comment_text=comment_text,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        labels=torch.FloatTensor(labels)
        )

In [ ]:
train_dataset = SensetiveData(train_df, tokenizer, MAX_TOKEN_COUNT)
test_dataset = SensetiveData(test_df, tokenizer, MAX_TOKEN_COUNT)

In [ ]:
class SensetiveDataModule(pl.LightningDataModule):
    def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
        super().__init__()

        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):

        self.train_dataset = SensetiveData(
        self.train_df,
        self.tokenizer,
        self.max_token_len
        )
        self.test_dataset = SensetiveData(
        self.test_df,
        self.tokenizer,
        self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=2
    )

    def val_dataloader(self):
        return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=2
    )

    def test_dataloader(self):
        return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=2
    )

In [ ]:
N_EPOCHS = 10
BATCH_SIZE = 12
data_module = SensetiveDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE, max_token_len=MAX_TOKEN_COUNT)

In [32]:

class SensetiveCommentTagger(pl.LightningModule):
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):

        output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        for i, name in enumerate(LABEL_COLUMNS):
            class_roc_auc = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
        )
        return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
            )
        )

In [33]:
dummy_model = nn.Linear(2, 1)
optimizer = torch.optim.AdamW(params=dummy_model.parameters(), lr=0.001)

steps_per_epoch=len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

model = SensetiveCommentTagger(
    n_classes=len(LABEL_COLUMNS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps
)

checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="sensetive-topics")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback, checkpoint_callback],
    max_epochs=N_EPOCHS,
    accelerator='gpu',
    devices=1
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 108 M 
1 | classifier | Linear    | 13.8 K
2 | criterion  | BCELoss   | 0     
-----------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.296   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 2465: 'val_loss' reached 0.18028 (best 0.18028), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 4930: 'val_loss' reached 0.09616 (best 0.09616), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 7395: 'val_loss' reached 0.07406 (best 0.07406), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 9860: 'val_loss' reached 0.06505 (best 0.06505), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [35]:
trained_model = SensetiveCommentTagger.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=len(LABEL_COLUMNS)
)
trained_model.eval()
trained_model.freeze()

IsADirectoryError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)
val_dataset = SensetiveData(
  val_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)
predictions = []
labels = []
for item in tqdm(val_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device),
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )

  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()
THRESHOLD = 0.5



  0%|          | 0/1557 [00:00<?, ?it/s]

In [ ]:
from torchmetrics.classification import MultilabelAccuracy
accuracy = MultilabelAccuracy(num_labels=18)

print(accuracy(predictions, labels))

tensor(0.9810)


In [31]:


input = { 
                'input_ids': torch.randint(BATCH_SIZE, [1,18],dtype=torch.long, device='cuda'),
                'attention_mask': torch.ones([1,18],dtype=torch.float, device='cuda'),
                'token_type_ids': torch.ones([1,18],dtype=torch.float, device='cuda')
              }
input_names = [ "text" ]
output_names = [ "offline_crime",
                "online_crime",
                "drugs","gambling",
                "pornography",
                "prostitution","slavery","suicide",
                "terrorism","weapons","body_shaming",
                "health_shaming","politics","racism","religion",
                "sexual_minorities","sexism","social_injustice" ]
symbolic_names = {0 : 'batch_size', 1 : 'max_seq_len'}

torch.onnx.export(trained_model,
                 (input['input_ids'],
                  input['attention_mask'],
                  input['token_type_ids']),
                 "sensetive-topics.onnx",
                 verbose=False,
                 opset_version=14,
                 input_names=['input_ids', 'input_mask', 'segment_ids'],
                 output_names=['start_logits', 'end_logits'],
                 export_params=True,
                 dynamic_axes={'input_ids' : symbolic_names,
                               'input_mask' : symbolic_names,
                               'segment_ids' : symbolic_names,
                               'start_logits' : symbolic_names,
                               'end_logits' : symbolic_names})

UnsupportedOperatorError: ignored

In [30]:
from google.colab import drive
drive.mount('/content/drive/folders/1TTty4rXecceWZ6p3nMy5wefqNtPqDn7c?ths=true')

ValueError: ignored

# New Section